In [1]:
from openai import OpenAI
import numpy as np

In [8]:
client = OpenAI(api_key="sk-proj-**************************************azvFPRdxjAA")

In [17]:
with open('messi.txt', "r") as f:
    text = f.read()
    print(text)

Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times being named the world's best player by FIFA.[note 2] He is the most decorated player in the history of professional football having won 45 team trophies,[note 3] including twelve league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most matches played (39), assists (18) and goal contributions (34) 

In [18]:
CHUNNK_SIZE = 100
chunks = []
current_place = 0
while current_place < len(text):
    current_chunk = text[current_place : current_place + CHUNNK_SIZE]
    chunks.append(current_chunk)
    current_place += CHUNNK_SIZE
print(chunks)

['Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), a', 'lso known as Leo Messi, is an Argentine professional footballer who plays as a forward for and capta', 'ins both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as on', 'e of the greatest players of all time, Messi set numerous records for individual accolades won throu', "ghout his professional footballing career such as eight Ballon d'Or awards and eight times being nam", "ed the world's best player by FIFA.[note 2] He is the most decorated player in the history of profes", 'sional football having won 45 team trophies,[note 3] including twelve league titles, four UEFA Champ', 'ions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European G', 'olden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (3', '6) and assists (192) in La Liga, most matches played (39), ass

In [19]:
embedded_chunks = [] # -> [ () , () , , ]
for chunk in chunks:
    response = client.embeddings.create(
        input=chunk,
        model="text-embedding-3-small"
    )
    embedded_chunks.append( (chunk, response.data[0].embedding) )
print(embedded_chunks)

[('Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), a', [0.024406712502241135, -0.043285056948661804, -0.014128047041594982, -0.0010167331201955676, -0.0006596927414648235, -0.00042102602310478687, 0.03548391908407211, 0.02796943672001362, -0.01705603301525116, -0.004699110984802246, -0.06285957247018814, 1.2567194971779827e-05, -0.004719586577266455, -0.024877646937966347, 0.026310928165912628, 0.03677387163043022, -0.06973931193351746, 0.0008375730249099433, -0.01692294329404831, -0.042793650180101395, 0.057658810168504715, -0.0038852128200232983, 0.0037470036186277866, 0.0026771624106913805, 0.01921619102358818, 0.007447937503457069, -0.0390261709690094, 0.003158335108309984, -0.011394577100872993, 0.02020924910902977, 0.07092688977718353, -0.0195130854845047, -0.0018018378177657723, -0.02066994644701481, 0.023771973326802254, 0.010616510175168514, -0.0011082326527684927, -0.014209948480129242, 0.0006686507840640843, -0.0309793259948

In [20]:
def cosine_simularity(A, B):
    return np.dot(A,B) / ( np.linalg.norm(A) * np.linalg.norm(B) )
    # ABCos / Vector A / |A| * B/|B|

In [21]:
question = "who is messi"

In [22]:
question_embedding = client.embeddings.create(
        input=question,
        model="text-embedding-3-small"
    ).data[0].embedding
print(question_embedding)

[0.007771440781652927, -0.07974173873662949, -0.062433116137981415, 0.006272740662097931, -0.05061791092157364, -0.02737988717854023, 0.03904249891638756, 0.030148394405841827, -0.02158128097653389, -0.05101029947400093, -0.03856291249394417, -0.01730862259864807, -0.023913802579045296, -0.02273664064705372, 0.02045861817896366, 0.04161481186747551, -0.0422687903046608, -0.018736474215984344, 0.0030273739248514175, -0.0027221841737627983, -0.005885803606361151, 0.03326569125056267, -0.008866854012012482, 0.009591680020093918, 0.017701009288430214, 0.03359267860651016, -0.011989600025117397, -0.007231908850371838, 0.020131628960371017, 0.049266356974840164, 0.03943488374352455, -0.04255218058824539, 0.026137327775359154, -0.02264944463968277, 0.009853271767497063, 0.013384753838181496, -0.0079022366553545, 0.0001437389582861215, -0.028317255899310112, -0.004643245134502649, -0.044950101524591446, -0.036709975451231, 0.0026308996602892876, 0.017112428322434425, 0.03470444306731224, -0.01

In [23]:
best_cosine_similarity = 0
index = 0
for i in range(len(embedded_chunks)):
    similarity = cosine_simularity(question_embedding, embedded_chunks[i][1])
    if similarity > best_cosine_similarity:
        best_cosine_similarity = similarity
        index = i

In [24]:
print(f"Relevant information: {embedded_chunks[index][0]}")

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user", "content": f"User question: {question}. Potentially useful information: {embedded_chunks[index][0]}"
        }
    ]
)

Relevant information: Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), a


In [25]:
print(completion.choices[0].message.content)

Lionel Andrés Messi, commonly known as Lionel Messi, is a professional soccer player from Argentina, widely regarded as one of the greatest footballers of all time. He was born on June 24, 1987, in Rosario, Argentina. Messi began his football career at a young age, and after showing prodigious talent, he moved to Spain to join FC Barcelona's famed youth academy, La Masia, at the age of 13.

Throughout his career at Barcelona, Messi set numerous records and won multiple prestigious awards, including winning the FIFA Ballon d'Or (awarded to the world's best player) multiple times. He is known for his exceptional dribbling, vision, and ability to score goals. Messi helped Barcelona win numerous domestic and international titles, including several UEFA Champions League and La Liga championships.

In 2021, Messi left Barcelona due to financial constraints at the club and joined Paris Saint-Germain (PSG) in France. He also captained the Argentina national team, leading them to victory in the